In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
#定義常用函數
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
#準備訓練資料 [:,np.newaxis]增加維度
x_data = np.linspace(0, 1, 400, dtype = np.double)[:,np.newaxis]

def judgeFunction(value):
    if value > 0.5:
        return 1
    return 0

In [ ]:
#學習速率
learning_rate = 0.001

def allShape(list):
    for arr in list:
        print(arr.shape)

In [ ]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(1, y) for y in sizes[1:]]
        self.weights = [np.random.randn(x, y) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, x):
        (xs, _) = self.feedforwardRecord(x)
        return xs[-1]
    
    def feedforwardRecord(self, x):
        xs = [np.array(x)]
        ss = [np.array(x)]
        
        for b, w in zip(self.biases, self.weights):
            s = np.dot(x, w)
            x = sigmoid(s + b)
            
            xs.append(x)
            ss.append(s)
        return (xs, ss)
        
    def backpropagationWith(self, xs, ss, y):
        
        delta_out = []
        delta_in = xs[-1] - y #cross entropy loss
        
        dbs = [delta_in]
        dws = [np.dot(np.transpose(xs[-1]), delta_in)]
        
        for idx in range(self.num_layers - 2, 0, -1):
            
            delta_out = np.dot(delta_in, np.transpose(self.weights[idx]))
            delta_in = delta_out * sigmoid_prime(ss[idx - 2])
            db = delta_in
            dw = np.dot(np.transpose(xs[idx - 1]), delta_in)
            
            dbs.insert(0, db)
            dws.insert(0, dw)
        
        return (dbs, dws)
    
    def updateWith(self, dbs, dws):
        
        new_b = []
        for b, db in zip(self.biases, dbs):
            new_b.append(b - learning_rate * db)
        
        new_w = []
        for w, dw in zip(self.weights, dws):
            new_w.append(w - learning_rate * dw)
        
        self.biases = new_b
        self.weights = new_w
        
    def showbiasesweights(self):
        allShape(self.biases)
        print(self.biases)
        allShape(self.weights)
        print(self.weights)

In [ ]:
myNN = Network([1, 2, 3, 1])
#print(myNN.biases, myNN.weights)

In [ ]:
for _ in range(1):
    idx = np.random.randint(x_data.size)
    (xs, ss) = myNN.feedforwardRecord(x_data[idx])
    (dbs, dws) = myNN.backpropagationWith(xs, ss, judgeFunction(idx))
    myNN.updateWith(dbs, dws)

In [ ]:
real_y = np.zeros(x_data.shape)
predict_y = np.zeros(x_data.shape)

index = 0
while index < x_data.size:
    real_y[index] = judgeFunction(np.full((1, 1), x_data[index]))
    predict_y[index] = myNN.feedforward(x_data[index])
    index = index + 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.plot(x_data, real_y,'b-', lw = 2) #真實
ax.plot(x_data, predict_y,'r-', lw = 2) #推測

plt.show()